In [6]:
import numpy as np
import pandas as pd
import subprocess
import requests
from time import sleep
from pathlib import Path
import datetime

In [7]:
datetime.datetime.now()

datetime.datetime(2021, 11, 7, 17, 31, 57, 332778)

In [8]:
def generate_intervals(initial, interval_size):
    intervals = []
    intervals.append((0, initial))
    for x in range(interval_size - 1):
        initial_interval_value = intervals[-1][1]
        interval_value = intervals[0][1]

        last_interval_value = initial_interval_value + interval_value
        intervals.append((initial_interval_value + 1, last_interval_value))

    return intervals

In [9]:
def build_table(min_pods, max_pods, initial_lat, interval_lat_size, initial_req, interval_req_size):
    requests_intervals = generate_intervals(initial_req, interval_req_size)

    table = []

    for request_interval in requests_intervals:
        latency_intervals = generate_intervals(initial_lat, interval_lat_size)
        for latency_interval in latency_intervals:
            for pod in range(min_pods, max_pods + 1):
                options = np.zeros(max_pods + 1 - min_pods)
                options = [0 for opt in range(len(options))]

                table.append([pod, latency_interval[0], latency_interval[1], request_interval[0], request_interval[1]] + list(options))

    labels = ['pod', 'initial_latency', 'end_latency', 'initial_request', 'end_request']
    actions = list(np.arange(min_pods, max_pods + 1).astype(np.str_))

    return pd.DataFrame(table, columns=labels+actions)

In [10]:
def find_options(table, pod, latency, request):
    return table[(table['pod'] == pod) & (table['initial_latency'] <= latency) & (table['end_latency'] >= latency) & (table['initial_request'] <= request) & (table['end_request'] >= request)].iloc[:,5::]

In [11]:
def find_best_q_value(table, pod, latency, request):
    print("find_best_q_value")
    options = find_options(table, pod, latency, request)
    
    return options.max(axis=1).item()

In [12]:
def find_q_value(table, pods, latency, requests, action):
    print("find_q_value")
    
    value =  table[(table['pod'] == pods) & (table['initial_latency'] <= latency) & (table['end_latency'] >= latency) & (table['initial_request'] <= requests) & (table['end_request'] >= requests)][str(action)]
    return value.item()

In [13]:
def find_best_action(table, pod, latency, request):
    print("find_best_action")
    options = find_options(table, pod, latency, request)
    
    return int(options.idxmax(axis=1))

In [14]:
def update_action_result(table, pod, latency, request, action, result):
    print("update_action_result")
    
    table.loc[(table['pod'] == pod) & (table['initial_latency'] <= latency) & (table['end_latency'] >= latency) & (table['initial_request'] <= request) & (table['end_request'] >= request), str(action)] = result
    print("updating pod", pod, " latency ", latency, " request ", request, " result ", result)
    return table

In [15]:
def get_latency():
    print("get_latency")
    query = 'rate(http_server_request_duration_seconds_sum{path="/test"}[30s])/rate(http_server_request_duration_seconds_count{path="/test"}[30s])'
    result = None

    while(True):
        response = requests.get("http://localhost:9090/api/v1/query?query={query}".format(query = query))
        if response.json()['status'] == 'success':
            result = response.json()['data']['result']
            if result == []:
                result = float(0)
            else:
                result = response.json()['data']['result'][0]['value'][1]
                if result == 'NaN': result = float(0)
            break
        sleep(5)
    
    return round(float(result) * 1000)

In [16]:
def get_request_by_second():
    print("get_request_by_second")    
    query = 'increase(http_server_requests_total{path="/test"}[30s])'
    result = None

    while(True):
        response = requests.get("http://localhost:9090/api/v1/query?query={query}".format(query = query))
        if response.json()['status'] == 'success':
            result = response.json()['data']['result']
            if result == []:
                result = float(0)
            else:
                result = response.json()['data']['result'][0]['value'][1]
                if result == 'NaN': result = float(0)
            break
        sleep(5)
        
    req = round(float(result))
    if req > 54:
        req = 54
    
    return req

In [17]:
def get_pods():
    print("get_pods")    
    # TODO comando pega quando esta como Terminating
    bashCommand = "kubectl get pods --field-selector=status.phase=Running"
    
    error = True
    output = None
    
    while(error == True):
        process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)
        output, error = process.communicate()
    
    return len(output.decode("utf-8").split('\n')[1:][:-1])  
    

In [18]:
def set_pods(new_pods):
    print("set_pods")
    bashCommand = "kubectl scale deployment.v1.apps/phpa-web-app-deployment --replicas={pods}".format(pods = new_pods)

    error = True
    output = None

    while(error == True):
        process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)
        output, error = process.communicate()

In [19]:
def run_q_learning(epochs, table, min_pods, max_pods):
    ideal_latency = 500
    alpha = 0.5
    gamma = 0.5
    epsilon = 0.1
    
    history_file = Path("_q_learning.csv")
    if history_file.is_file():
        history = pd.read_csv("_q_learning.csv")
    else:
        history = pd.DataFrame([], columns=['datetime','epoch','pod', 'latency', 'requests', 'action', 'new_latency', 'new_requests', 'reward', 'q_value', 'random'])      
    
    latency = get_latency()
    pods = get_pods()
    requests = get_request_by_second()

    for x in range(epochs):
        print("\n--- Exec ", x,"\n\n latency ", latency, "\n pods ", pods, "\n requests ", requests)
        is_random = None

        if np.random.uniform(0, 1) > epsilon:
            action = find_best_action(table, pods, latency, requests)
            is_random = False
            print("choose action ", action)
        else:
            action = np.random.randint(min_pods, max_pods + 1)
            is_random = True
            print("random ", action)

        set_pods(action)
        sleep(45)

        new_latency = get_latency()
        new_requests = get_request_by_second()
        reward = get_reward(ideal_latency, pods, action, latency, new_latency)
        print(" action ", action, "\n new_latency ", new_latency, "\n new_requests ", new_requests, "\n reward ", reward)

        new_value = new_q_value(table, pods, latency, requests, action, new_latency, new_requests, reward, alpha, gamma)
        print("*** new value: ", new_value, " ***")
        
        table = update_action_result(table, pods, latency, requests, action, new_value)
        values = {'datetime': datetime.datetime.now(),'epoch': x,'pod': pods, 'latency': latency, 'requests': requests, 'action': action, 'new_latency': new_latency, 'new_requests': new_requests, 'reward': reward, 'q_value' : new_value, 'random': is_random}
        history = history.append(values, ignore_index=True)
        # setting new state
        latency = new_latency
        pods = action
        requests = new_requests
    
    table.to_csv('table_q_learning.csv', index=False)
    history.to_csv('history_q_learning.csv', index=False)
    print("acabou: ", datetime.datetime.now())

In [20]:
def new_q_value(table, pods, latency, requests, action, new_latency, new_requests, reward, alpha, gamma):
    old_value = find_q_value(table, pods, latency, requests, action)
    next_max = find_best_q_value(table, action, new_latency, new_requests)

    return (1 - alpha) * old_value + alpha * (reward + gamma * next_max)

In [21]:
def is_ideal_latency(ideal_latency, latency):
    print("is_ideal_latency")
    return latency <= ideal_latency

In [22]:
def obtained_result(new_value, old_value):
    print("obtained_result")    
    if new_value == old_value:
        return 'kept'
    elif new_value > old_value:
        return 'increased'
    else:
        return 'decreased'

In [23]:
def reward_within_ideal(pod_state, latency_state, new_latency_is_ideal):
    print("reward_within_ideal")
    if not new_latency_is_ideal:
        return -20 # qualquer ação que tomou resultou em levar a latencia para acima do desejado
    
    if pod_state == 'kept': return 10 # não tomou nenhuma ação, mas manteve a latencia ideal
    elif pod_state == 'increased':
        return {
            'kept': -10, # aumentou pods e a latencia se manteve - pode ter aumentado as requisições
            'increased': 0, # aumentou pods e a latencia aumentou - pode ter aumentado as requisições
            'decreased': -20 # ja estava na latencia ideal e aumentou pods desnecessáriamente
        }.get(latency_state)
    else: # pod_state == 'decreased'
        return {
            'kept': 20, # diminiu pods e a latencia se manteve
            'increased': 20, # diminiu pods, latencia aumentou mas segue estando dentro do desejado: ótimo cenário
            'decreased': 20 # diminiu pods, diminiu latencia - pode ter diminuido as requisições
        }.get(latency_state)

In [24]:
def reward_out_ideal(pod_state, latency_state, new_latency_is_ideal):
    print("reward_out_ideal")    
    if new_latency_is_ideal:
        return 20 # qualquer ação tomada resultou em levar a latencia para dentro do desejado
    
    if pod_state == 'kept':
        return {
            'kept': -20, # não tomou nenhuma ação
            'increased': -20, # não tomou nenhuma ação e aumentou a latencua
            'decreased': -10 # não tomou nenhuma ação mas diminiu a latencia - pode ter diminuido requisições
        }.get(latency_state)
    elif pod_state == 'increased':
        return {
            'kept': 0, # aumentou pods e manteve a latencia - podem ter aumentado as requisições
            'increased': 0, # aumentou pods e aumentou latencia - podem ter aumentado as requisições
            'decreased': 10 # aumentou pods e diminiu a latencia
        }.get(latency_state)
    else: # pod_state == 'decreased'
        return {
            'kept': -10, # diminiu pods e a latencia se manteve
            'increased': -20, # diminiu pods, latencia aumentou
            'decreased': 0 # diminiu pods, diminiu latencia - pode ter diminuido as requisições
        }.get(latency_state)

In [25]:
def get_reward(ideal_latency, old_pods, new_pods, old_latency, new_latency):
    print("get_reward")    
    pod_state = obtained_result(new_pods, old_pods)
    latency_state = obtained_result(new_latency, old_latency)

    old_latency_is_ideal = is_ideal_latency(ideal_latency, old_latency)
    new_latency_is_ideal = is_ideal_latency(ideal_latency, new_latency)

    reward = 0
    if old_latency_is_ideal:
        reward = reward_within_ideal(pod_state, latency_state, new_latency_is_ideal)
    else:
        reward = reward_out_ideal(pod_state, latency_state, new_latency_is_ideal)

    return reward

In [26]:
# def run_sarsa(epochs, table, min_pods, max_pods):
#     ideal_latency = 500
#     alpha = 0.1
#     gamma = 0.6
#     epsilon = 0.1
    
#     history_file = Path("history_sarsa.csv")
#     if history_file.is_file():
#         history = pd.read_csv("history_sarsa.csv")
#     else:
#         history = pd.DataFrame([], columns=['datetime', 'epoch','pod', 'latency', 'requests', 'action', 'new_latency', 'new_requests', 'reward', 'q_value', 'random'])      
    
#     latency = get_latency()
#     pods = get_pods()
#     requests = get_request_by_second()

#     for x in range(epochs):
#         print("\n--- Exec ", x,"\n\n latency ", latency, "\n pods ", pods, "\n requests ", requests)
#         is_random = None

#         if np.random.uniform(0, 1) > epsilon:
#             action = find_best_action(table, pods, latency, requests)
#             is_random = False
#             print("choose action ", action)
#         else:
#             action = np.random.randint(min_pods, max_pods + 1)
#             is_random = True
#             print("random ", action)

#         set_pods(action)
#         sleep(45)

#         new_latency = get_latency()
#         new_requests = get_request_by_second()
#         reward = get_reward(ideal_latency, pods, action, latency, new_latency)
#         print(" action ", action, "\n new_latency ", new_latency, "\n new_requests ", new_requests, "\n reward ", reward)

#         new_value = new_value(table, pods, latency, requests, action, new_latency, new_requests, reward, alpha, gamma)
#         print("*** new value: ", new_value, " ***")
        
#         table = update_action_result(table, pods, latency, requests, action, new_value)
#         values = {'datetime': datetime.datetime.now(),'epoch': x,'pod': pods, 'latency': latency, 'requests': requests, 'action': action, 'new_latency': new_latency, 'new_requests': new_requests, 'reward': reward, 'q_value' : new_value, 'random': is_random}
#         history = history.append(values, ignore_index=True)
#         # setting new state
#         latency = new_latency
#         pods = action
#         requests = new_requests
    
#     table.to_csv('table_sarsa.csv', index=False)
#     history.to_csv('history_sarsa.csv', index=False)
#     print("acabou: ", datetime.datetime.now())

In [27]:
def run_sarsa(epochs, table, min_pods, max_pods):
    ideal_latency = 500
    alpha = 0.5
    gamma = 0.5
    epsilon = 0.1
    
    history_file = Path("history_sarsa.csv")
    if history_file.is_file():
        history = pd.read_csv("history_sarsa.csv")
    else:
        history = pd.DataFrame([], columns=['datetime', 'epoch','pod', 'latency', 'requests', 'action', 'next_action', 'new_latency', 'new_requests', 'reward', 'q_value', 'random'])      
    
    # get state
    latency = get_latency()
    pods = get_pods()
    requests = get_request_by_second()
    
    is_random = None
    action = None
    
    # choose action
    if np.random.uniform(0, 1) > epsilon:
        action = find_best_action(table, pods, latency, requests)
        is_random = False
        print("choose action ", action)
    else:
        action = np.random.randint(min_pods, max_pods + 1)
        is_random = True
        print("random ", action)

    for x in range(epochs):
        print("\n--- Exec ", x,"\n\n latency ", latency, "\n pods ", pods, "\n requests ", requests)
        
        # take action
        set_pods(action)
        sleep(45)
        
        # get new state
        new_latency = get_latency()
        new_requests = get_request_by_second()
        reward = get_reward(ideal_latency, pods, action, latency, new_latency)
        print("action ", action, "\n new_latency ", new_latency, "\n new_requests ", new_requests, "\n reward ", reward)

        # choose next action
        next_action = None
        if np.random.uniform(0, 1) > epsilon:
            next_action = find_best_action(table, action, new_latency, new_requests)
            is_random = False
            print("choose next_action action ", next_action)
        else:
            next_action = np.random.randint(min_pods, max_pods + 1)
            is_random = True
            print("random next_action", next_action)
            
        # update q-value
        updated_value = new_value(table, pods, latency, requests, action, new_latency, new_requests, reward, alpha, gamma, next_action)
        print("*** new value: ", updated_value, " ***")
        table = update_action_result(table, pods, latency, requests, action, updated_value)
        
        values = {'datetime': datetime.datetime.now(),'epoch': x,'pod': pods, 'latency': latency, 'requests': requests, 'action': action, 'next_action': next_action, 'new_latency': new_latency, 'new_requests': new_requests, 'reward': reward, 'q_value' : updated_value, 'random': is_random}
        history = history.append(values, ignore_index=True)
        
        # setting new state and action
        latency = new_latency
        pods = action
        requests = new_requests
        action = next_action
    
    table.to_csv('table_sarsa.csv', index=False)
    history.to_csv('history_sarsa.csv', index=False)
    print("acabou: ", datetime.datetime.now())

In [28]:
def new_value(table, pods, latency, requests, action, new_latency, new_requests, reward, alpha, gamma, next_action):
    old_value = find_q_value(table, pods, latency, requests, action)
    next_value = find_q_value(table, action, new_latency, new_requests, next_action)

    return (1 - alpha) * old_value + alpha * (reward + gamma * next_value)

In [29]:
def exec_q_learning():
    epochs = 1800
    min_pods = 1
    max_pods = 3
    initial_latency = 500
    interval_size = 4
    initial_request = 9
    interval_request = 6
    
    table_file = Path("table_q_learning.csv")
    if table_file.is_file():
        table = pd.read_csv("table_q_learning.csv")
    else:
        table = build_table(min_pods, max_pods, initial_latency, interval_size, initial_request, interval_request)

    print("table: \n", table)
    print("Start at: ", datetime.datetime.now())
    
    run_q_learning(epochs, table, min_pods, max_pods)

In [30]:
def exec_sarsa():
    epochs = 1800
    min_pods = 1
    max_pods = 3
    initial_latency = 500
    interval_size = 4
    initial_request = 9
    interval_request = 6
    
    table_file = Path("table_sarsa.csv")
    if table_file.is_file():
        table = pd.read_csv("table_sarsa.csv")
    else:
        table = build_table(min_pods, max_pods, initial_latency, interval_size, initial_request, interval_request)

    print("table: \n", table)
    print("Start at: ", datetime.datetime.now())
    
    run_sarsa(epochs, table, min_pods, max_pods)

In [26]:
# run(epochs, table, min_pods, max_pods)

In [121]:
pd.set_option('display.max_rows', None)

In [122]:
def coisa(epochs, pods):
    without_phpa = pd.DataFrame([], columns=['epoch','pod', 'latency', 'requests'])      

    for x in range(epochs):
        latency = get_latency()
        pods = get_pods()
        requests = get_request_by_second()
        
        print("\n--- Exec ", x,"\n\n latency ", latency, "\n pods ", pods, "\n requests ", requests)
        
        values = {'epoch': x,'pod': pods, 'latency': latency, 'requests': requests}
        without_phpa = without_phpa.append(values, ignore_index=True)
        
        sleep(45)

    without_phpa.to_csv('without_phpa.csv', index=False)
    print("acabou: ", datetime.datetime.now())

In [65]:
# coisa(2100, 2)

In [28]:
exec_q_learning()

table: 
     pod  initial_latency  end_latency  initial_request  end_request  1  2  3
0     1                0          500                0            9  0  0  0
1     2                0          500                0            9  0  0  0
2     3                0          500                0            9  0  0  0
3     1              501         1000                0            9  0  0  0
4     2              501         1000                0            9  0  0  0
..  ...              ...          ...              ...          ... .. .. ..
67    2             1001         1500               46           54  0  0  0
68    3             1001         1500               46           54  0  0  0
69    1             1501         2000               46           54  0  0  0
70    2             1501         2000               46           54  0  0  0
71    3             1501         2000               46           54  0  0  0

[72 rows x 8 columns]
Start at:  2021-11-02 19:17:29.885913
get_la

get_latency
get_request_by_second
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal_latency
reward_within_ideal
 action  1 
 new_latency  309 
 new_requests  8 
 reward  10
find_q_value
find_best_q_value
*** new value:  17.45391845703125  ***
update_action_result
updating pod 1  latency  261  request  8  result  17.45391845703125

--- Exec  18 

 latency  309 
 pods  1 
 requests  8
find_best_action
choose action  1
set_pods
get_latency
get_request_by_second
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal_latency
reward_within_ideal
 action  1 
 new_latency  284 
 new_requests  8 
 reward  10
find_q_value
find_best_q_value
*** new value:  18.090438842773438  ***
update_action_result
updating pod 1  latency  309  request  8  result  18.090438842773438

--- Exec  19 

 latency  284 
 pods  1 
 requests  8
find_best_action
choose action  1
set_pods
get_latency
get_request_by_second
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal

get_latency
get_request_by_second
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal_latency
reward_within_ideal
 action  1 
 new_latency  276 
 new_requests  6 
 reward  10
find_q_value
find_best_q_value
*** new value:  19.66632811657071  ***
update_action_result
updating pod 1  latency  258  request  8  result  19.66632811657071

--- Exec  37 

 latency  276 
 pods  1 
 requests  6
find_best_action
choose action  1
set_pods
get_latency
get_request_by_second
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal_latency
reward_within_ideal
 action  1 
 new_latency  279 
 new_requests  6 
 reward  10
find_q_value
find_best_q_value
*** new value:  19.749746087428033  ***
update_action_result
updating pod 1  latency  276  request  6  result  19.749746087428033

--- Exec  38 

 latency  279 
 pods  1 
 requests  6
find_best_action
choose action  1
set_pods
get_latency
get_request_by_second
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal

get_latency
get_request_by_second
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal_latency
reward_within_ideal
 action  1 
 new_latency  276 
 new_requests  6 
 reward  10
find_q_value
find_best_q_value
*** new value:  19.97177664796311  ***
update_action_result
updating pod 1  latency  208  request  6  result  19.97177664796311

--- Exec  56 

 latency  276 
 pods  1 
 requests  6
find_best_action
choose action  1
set_pods
get_latency
get_request_by_second
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal_latency
reward_within_ideal
 action  1 
 new_latency  208 
 new_requests  4 
 reward  10
find_q_value
find_best_q_value
*** new value:  19.978832485972333  ***
update_action_result
updating pod 1  latency  276  request  6  result  19.978832485972333

--- Exec  57 

 latency  208 
 pods  1 
 requests  4
find_best_action
choose action  1
set_pods
get_latency
get_request_by_second
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal

get_latency
get_request_by_second
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal_latency
reward_within_ideal
 action  1 
 new_latency  207 
 new_requests  6 
 reward  10
find_q_value
find_best_q_value
*** new value:  19.988246328024687  ***
update_action_result
updating pod 1  latency  289  request  10  result  19.988246328024687

--- Exec  75 

 latency  207 
 pods  1 
 requests  6
find_best_action
choose action  1
set_pods
get_latency
get_request_by_second
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal_latency
reward_within_ideal
 action  1 
 new_latency  279 
 new_requests  6 
 reward  10
find_q_value
find_best_q_value
*** new value:  19.99021625461785  ***
update_action_result
updating pod 1  latency  207  request  6  result  19.99021625461785

--- Exec  76 

 latency  279 
 pods  1 
 requests  6
find_best_action
choose action  1
set_pods
get_latency
get_request_by_second
get_reward
obtained_result
obtained_result
is_ideal_latency
is_idea

...

get_latency
get_request_by_second
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal_latency
reward_out_ideal
 action  3 
 new_latency  576 
 new_requests  22 
 reward  -20
find_q_value
find_best_q_value
*** new value:  -16.022573788734277  ***
update_action_result
updating pod 3  latency  556  request  24  result  -16.022573788734277

--- Exec  1707 

 latency  576 
 pods  3 
 requests  22
find_best_action
choose action  1
set_pods
get_latency
get_request_by_second
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal_latency
reward_out_ideal
 action  1 
 new_latency  1325 
 new_requests  48 
 reward  -20
find_q_value
find_best_q_value
*** new value:  -13.377525600378458  ***
update_action_result
updating pod 3  latency  576  request  22  result  -13.377525600378458

--- Exec  1708 

 latency  1325 
 pods  1 
 requests  48
find_best_action
choose action  2
set_pods
get_latency
get_request_by_second
get_reward
obtained_result
obtained_result
is_ideal_la

get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal_latency
reward_out_ideal
 action  3 
 new_latency  544 
 new_requests  20 
 reward  10
find_q_value
find_best_q_value
*** new value:  5.955114497829247  ***
update_action_result
updating pod 2  latency  718  request  28  result  5.955114497829247

--- Exec  1725 

 latency  544 
 pods  3 
 requests  20
find_best_action
choose action  1
set_pods
get_latency
get_request_by_second
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal_latency
reward_out_ideal
 action  1 
 new_latency  1256 
 new_requests  46 
 reward  -20
find_q_value
find_best_q_value
*** new value:  -13.003007582966854  ***
update_action_result
updating pod 3  latency  544  request  20  result  -13.003007582966854

--- Exec  1726 

 latency  1256 
 pods  1 
 requests  46
find_best_action
choose action  2
set_pods
get_latency
get_request_by_second
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal_latency
reward_out_ideal

get_latency
get_request_by_second
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal_latency
reward_within_ideal
 action  2 
 new_latency  207 
 new_requests  2 
 reward  10
find_q_value
find_best_q_value
*** new value:  16.06757169679117  ***
update_action_result
updating pod 2  latency  217  request  4  result  16.06757169679117

--- Exec  1744 

 latency  207 
 pods  2 
 requests  2
find_best_action
choose action  2
set_pods
get_latency
get_request_by_second
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal_latency
reward_within_ideal
 action  2 
 new_latency  294 
 new_requests  6 
 reward  10
find_q_value
find_best_q_value
*** new value:  17.05067877259338  ***
update_action_result
updating pod 2  latency  207  request  2  result  17.05067877259338

--- Exec  1745 

 latency  294 
 pods  2 
 requests  6
find_best_action
choose action  2
set_pods
get_latency
get_request_by_second
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ide

get_latency
get_request_by_second
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal_latency
reward_within_ideal
 action  1 
 new_latency  210 
 new_requests  6 
 reward  10
find_q_value
find_best_q_value
*** new value:  18.883730132754792  ***
update_action_result
updating pod 1  latency  289  request  10  result  18.883730132754792

--- Exec  1763 

 latency  210 
 pods  1 
 requests  6
find_best_action
choose action  1
set_pods
get_latency
get_request_by_second
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal_latency
reward_within_ideal
 action  1 
 new_latency  290 
 new_requests  10 
 reward  10
find_q_value
find_best_q_value
*** new value:  18.921573789798973  ***
update_action_result
updating pod 1  latency  210  request  6  result  18.921573789798973

--- Exec  1764 

 latency  290 
 pods  1 
 requests  10
random  2
set_pods
get_latency
get_request_by_second
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal_latency
reward

get_latency
get_request_by_second
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal_latency
reward_within_ideal
 action  1 
 new_latency  207 
 new_requests  6 
 reward  10
find_q_value
find_best_q_value
*** new value:  19.882270954907582  ***
update_action_result
updating pod 1  latency  259  request  8  result  19.882270954907582

--- Exec  1782 

 latency  207 
 pods  1 
 requests  6
find_best_action
choose action  1
set_pods
get_latency
get_request_by_second
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal_latency
reward_within_ideal
 action  1 
 new_latency  289 
 new_requests  10 
 reward  10
find_q_value
find_best_q_value
*** new value:  19.869094688359585  ***
update_action_result
updating pod 1  latency  207  request  6  result  19.869094688359585

--- Exec  1783 

 latency  289 
 pods  1 
 requests  10
find_best_action
choose action  1
set_pods
get_latency
get_request_by_second
get_reward
obtained_result
obtained_result
is_ideal_latency


In [31]:
exec_sarsa()

table: 
     pod  initial_latency  end_latency  initial_request  end_request  1  2  3
0     1                0          500                0            9  0  0  0
1     2                0          500                0            9  0  0  0
2     3                0          500                0            9  0  0  0
3     1              501         1000                0            9  0  0  0
4     2              501         1000                0            9  0  0  0
..  ...              ...          ...              ...          ... .. .. ..
67    2             1001         1500               46           54  0  0  0
68    3             1001         1500               46           54  0  0  0
69    1             1501         2000               46           54  0  0  0
70    2             1501         2000               46           54  0  0  0
71    3             1501         2000               46           54  0  0  0

[72 rows x 8 columns]
Start at:  2021-11-07 17:32:16.947119
get_la

get_latency
get_request_by_second
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal_latency
reward_within_ideal
action  1 
 new_latency  262 
 new_requests  8 
 reward  10
find_best_action
choose next_action action  1
find_q_value
find_q_value
*** new value:  17.929699420928955  ***
update_action_result
updating pod 1  latency  206  request  6  result  17.929699420928955

--- Exec  17 

 latency  262 
 pods  1 
 requests  8
set_pods
get_latency
get_request_by_second
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal_latency
reward_within_ideal
action  1 
 new_latency  287 
 new_requests  8 
 reward  10
find_best_action
choose next_action action  1
find_q_value
find_q_value
*** new value:  18.447274565696716  ***
update_action_result
updating pod 1  latency  262  request  8  result  18.447274565696716

--- Exec  18 

 latency  287 
 pods  1 
 requests  8
set_pods
get_latency
get_request_by_second
get_reward
obtained_result
obtained_result
is_ideal_la

get_latency
get_request_by_second
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal_latency
reward_within_ideal
action  1 
 new_latency  259 
 new_requests  8 
 reward  10
random next_action 1
find_q_value
find_q_value
*** new value:  15.932381671396662  ***
update_action_result
updating pod 1  latency  311  request  8  result  15.932381671396662

--- Exec  36 

 latency  259 
 pods  1 
 requests  8
set_pods
get_latency
get_request_by_second
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal_latency
reward_within_ideal
action  1 
 new_latency  298 
 new_requests  10 
 reward  10
find_best_action
choose next_action action  1
find_q_value
find_q_value
*** new value:  17.694196770147443  ***
update_action_result
updating pod 1  latency  259  request  8  result  17.694196770147443

--- Exec  37 

 latency  298 
 pods  1 
 requests  10
set_pods
get_latency
get_request_by_second
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal_latency


get_request_by_second
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal_latency
reward_within_ideal
action  1 
 new_latency  206 
 new_requests  6 
 reward  10
find_best_action
choose next_action action  1
find_q_value
find_q_value
*** new value:  19.860442520962295  ***
update_action_result
updating pod 1  latency  258  request  8  result  19.860442520962295

--- Exec  54 

 latency  206 
 pods  1 
 requests  6
set_pods
get_latency
get_request_by_second
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal_latency
reward_within_ideal
action  1 
 new_latency  289 
 new_requests  10 
 reward  10
find_best_action
choose next_action action  1
find_q_value
find_q_value
*** new value:  19.881708275407437  ***
update_action_result
updating pod 1  latency  206  request  6  result  19.881708275407437

--- Exec  55 

 latency  289 
 pods  1 
 requests  10
set_pods
get_latency
get_request_by_second
get_reward
obtained_result
obtained_result
is_ideal_latency
is_i

get_latency
get_request_by_second
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal_latency
reward_within_ideal
action  1 
 new_latency  271 
 new_requests  6 
 reward  10
find_best_action
choose next_action action  1
find_q_value
find_q_value
*** new value:  17.1730390463971  ***
update_action_result
updating pod 1  latency  207  request  6  result  17.1730390463971

--- Exec  73 

 latency  271 
 pods  1 
 requests  6
set_pods
get_latency
get_request_by_second
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal_latency
reward_within_ideal
action  1 
 new_latency  275 
 new_requests  6 
 reward  10
find_best_action
choose next_action action  1
find_q_value
find_q_value
*** new value:  17.879779284797827  ***
update_action_result
updating pod 1  latency  271  request  6  result  17.879779284797827

--- Exec  74 

 latency  275 
 pods  1 
 requests  6
set_pods
get_latency
get_request_by_second
get_reward
obtained_result
obtained_result
is_ideal_latenc

...

get_latency
get_request_by_second
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal_latency
reward_within_ideal
action  1 
 new_latency  276 
 new_requests  6 
 reward  10
find_best_action
choose next_action action  1
find_q_value
find_q_value
*** new value:  17.194095506066944  ***
update_action_result
updating pod 1  latency  311  request  8  result  17.194095506066944

--- Exec  1726 

 latency  276 
 pods  1 
 requests  6
set_pods
get_latency
get_request_by_second
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal_latency
reward_within_ideal
action  1 
 new_latency  207 
 new_requests  4 
 reward  10
find_best_action
choose next_action action  1
find_q_value
find_q_value
*** new value:  17.895571629550208  ***
update_action_result
updating pod 1  latency  276  request  6  result  17.895571629550208

--- Exec  1727 

 latency  207 
 pods  1 
 requests  4
set_pods
get_latency
get_request_by_second
get_reward
obtained_result
obtained_result
is_idea

get_latency
get_request_by_second
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal_latency
reward_within_ideal
action  2 
 new_latency  276 
 new_requests  6 
 reward  20
random next_action 2
find_q_value
find_q_value
*** new value:  15.967611877739174  ***
update_action_result
updating pod 3  latency  207  request  2  result  15.967611877739174

--- Exec  1745 

 latency  276 
 pods  2 
 requests  6
set_pods
get_latency
get_request_by_second
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal_latency
reward_within_ideal
action  2 
 new_latency  207 
 new_requests  6 
 reward  10
find_best_action
choose next_action action  2
find_q_value
find_q_value
*** new value:  14.103341806880238  ***
update_action_result
updating pod 2  latency  276  request  6  result  14.103341806880238

--- Exec  1746 

 latency  207 
 pods  2 
 requests  6
set_pods
get_latency
get_request_by_second
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal_latenc

get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal_latency
reward_within_ideal
action  2 
 new_latency  207 
 new_requests  4 
 reward  10
find_best_action
choose next_action action  2
find_q_value
find_q_value
*** new value:  19.851127175665845  ***
update_action_result
updating pod 2  latency  309  request  4  result  19.851127175665845

--- Exec  1763 

 latency  207 
 pods  2 
 requests  4
set_pods
get_latency
get_request_by_second
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal_latency
reward_within_ideal
action  2 
 new_latency  210 
 new_requests  6 
 reward  10
find_best_action
choose next_action action  2
find_q_value
find_q_value
*** new value:  19.88834538174938  ***
update_action_result
updating pod 2  latency  207  request  4  result  19.88834538174938

--- Exec  1764 

 latency  210 
 pods  2 
 requests  6
set_pods
get_latency
get_request_by_second
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal_latency
reward_wi

get_latency
get_request_by_second
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal_latency
reward_within_ideal
action  2 
 new_latency  207 
 new_requests  2 
 reward  10
find_best_action
choose next_action action  2
find_q_value
find_q_value
*** new value:  16.37795420583943  ***
update_action_result
updating pod 2  latency  0  request  0  result  16.37795420583943

--- Exec  1782 

 latency  207 
 pods  2 
 requests  2
set_pods
get_latency
get_request_by_second
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal_latency
reward_within_ideal
action  2 
 new_latency  213 
 new_requests  2 
 reward  10
find_best_action
choose next_action action  2
find_q_value
find_q_value
*** new value:  17.283465654379572  ***
update_action_result
updating pod 2  latency  207  request  2  result  17.283465654379572

--- Exec  1783 

 latency  213 
 pods  2 
 requests  2
set_pods
get_latency
get_request_by_second
get_reward
obtained_result
obtained_result
is_ideal_la

get_request_by_second
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal_latency
reward_within_ideal
action  2 
 new_latency  207 
 new_requests  4 
 reward  10
find_best_action
choose next_action action  2
find_q_value
find_q_value
*** new value:  19.979579955794122  ***
update_action_result
updating pod 2  latency  207  request  4  result  19.979579955794122
acabou:  2021-11-08 17:06:19.673955
